Learning F# 

In [55]:
let generateSquare (min:int) (max:int): int seq = 
    Seq.append {(max-1)..(-1)..min} {(min+1)..max}
    |> (fun x -> seq { for i in 1..4 do yield! x } )

let rec squareBuilder ((goal, acc, min, max):int*int*int*int) : int seq =
    match (goal <= acc) with 
    | true -> Seq.empty
    | _ -> 
        let newSquare = generateSquare min max
        squareBuilder (goal, (acc + Seq.length newSquare), (min+1), (max+2))
        |> Seq.append newSquare
    
let adventOfCode3Part1 (goal:int) : int = 
   
    squareBuilder (goal, 0, 1, 2)
    |> Seq.toList
    |> (fun distanceList -> distanceList.[goal-2])
    


In [56]:
adventOfCode3Part1 277678

475

In [57]:
squareBuilder (100, 0, 1, 2) |> Seq.toList

[1; 2; 1; 2; 1; 2; 1; 2; 3; 2; 3; 4; 3; 2; 3; 4; 3; 2; 3; 4; 3; 2; 3; 4; 5; 4; 3;
 4; 5; 6; 5; 4; 3; 4; 5; 6; 5; 4; 3; 4; 5; 6; 5; 4; 3; 4; 5; 6; 7; 6; 5; 4; 5; 6;
 7; 8; 7; 6; 5; 4; 5; 6; 7; 8; 7; 6; 5; 4; 5; 6; 7; 8; 7; 6; 5; 4; 5; 6; 7; 8; 9;
 8; 7; 6; 5; 6; 7; 8; 9; 10; 9; 8; 7; 6; 5; 6; 7; 8; 9; 10; ...]

In [58]:
type Movement = Up | Left | Down | Right

In [59]:
let advanceMovement (movement:Movement) : Movement =
    match movement with
    | Up -> Left
    | Left -> Down
    | Down -> Right
    | Right -> Up
    

In [60]:
type Point = int*int

In [80]:
type Item = Point*int

In [62]:
let getEnd (data:seq<'T>) : 'T = 
    Seq.rev data
    |> Seq.head   

In [81]:
let getScore (data:Item) : int =
    data |> (fun (_, num) -> num)

let getCoords (data:Item) : Point = 
    data |> (fun (point, _) -> point)

In [64]:
let makeSurroundingPoints ((x, y):Point) : Point seq = 
    seq [(x+1, y); (x+1, y+1); (x, y+1); (x-1, y+1); (x-1, y); (x-1, y-1); (x, y-1); (x+1, y-1);]

In [83]:
let findVal (point:Point) (item:Item) : bool =
    point
    |> makeSurroundingPoints
    |> Seq.exists (fun x -> (getCoords item) = x)


In [84]:
let getVal ((x, y):Point) (items:Item seq) : int = 
    items
    |> Seq.filter (findVal (x, y))
    |> Seq.map (fun (_, num) -> num)
    |> Seq.sum

In [85]:
let updatePoint (movement:Movement) (point:Point) : Point =
    match movement with
    | Up -> point |> (fun (x, y) -> x, y+1)
    | Left -> point |> (fun (x, y) -> x-1, y)
    | Down -> point |> (fun (x, y) -> x, y-1)
    | Right -> point |> (fun (x, y) -> x+1, y)

In [86]:
let addNewItem (current:Item seq) (newMove:Movement) : Item = 
    let lastPoint = getEnd current
    let lastCoords = getCoords lastPoint
    let newCoords = updatePoint newMove lastCoords
    let newVal = getVal newCoords current
    (newCoords, newVal)
    

In [91]:
let getNewParams (direction:Movement) (repeat:int) (advancement:int) : (Movement*int*int) =
    if repeat = advancement && (direction = Down || direction = Up) then
        ((advanceMovement direction), repeat + 1, 1)
    elif repeat = advancement && (direction = Right || direction = Left) then
        ((advanceMovement direction), repeat, 1)
    else
        (direction, repeat, advancement + 1)
        

In [92]:
let rec adventOfCode3Part2 (limit:int)  (direction:Movement) (repeat:int) (advancement:int) (current:Item seq) : int =
    if (getEnd >> getScore) current >= limit then
        current
        |> getEnd
        |> getScore
    else
        let newItem = addNewItem current direction
        let newList = newItem |> Seq.singleton |> Seq.append current
        
        getNewParams direction repeat advancement 
        |> (fun (newDir, newRep, newAdv) -> adventOfCode3Part2 limit newDir newRep newAdv newList)       
        

In [95]:
adventOfCode3Part2 277678 Right 1 1 (Seq.singleton ((0,0), 1))

279138